In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Descenso de gradiente

Hasta ahora, hemos visto el descenso de gradiente como un algoritmo de propósito general para optimizar la pérdida de entrenamiento.

$$\text{TrainLoss}(\mathbf{w}) =  \frac{1}{|\mathcal{D}_{\text{train}}|} \sum_{(x, y)\in\mathcal{D}_{\text{train}}} \text{Loss}(x, y, \mathbf{w})$$



1. Inicializa $\mathbf{w} = [0, \dots, 0]$
2. Para $t = 1, \dots, T$:
    1. $\mathbf{w} \gets \mathbf{w} - \eta \nabla_{\mathbf{w}} \mathrm{TrainLoss}(\mathbf{w})$

Un problema con el descenso de gradiente es que es lento: ¡Cada iteración requiere revisar todos los ejemplos de entrenamiento!

La pérdida de entrenamiento considera **todos** los ejemplos del conjunto de datos de entrenamiento. Si tenemos un millón de ejemplos, el cálculo del gradiente requerirá iterar sobre todo este millón, y esto ocurre en cada época.

¿Podemos progresar en nuestra búsqueda del mejor predictor antes de ver todos los datos?

# Descenso de gradiente estocástico

En lugar de iterar sobre todos los ejemplos de entrenamiento para calcular un solo gradiente y avanzar un paso, el *descenso de gradiente estocástico* (SGD) itera por los ejemplos $(x, y)$ y actualiza los pesos $\mathbf{w}$ con base en **cada** ejemplo.

1. Inicializa $\mathbf{w} = [0, \dots, 0]$
2. Para $t = 1, \dots, T$:
    1. Para $(x, y)\in\mathcal{D}_{\text{train}}$:
       1. $\mathbf{w} \gets \mathbf{w} - \eta \nabla_{\mathbf{w}} \mathrm{Loss}(x, y, \mathbf{w})$
      

Cada actualización a los pesos no es tan buena porque solo estamos considerando un ejemplo en lugar de todos. ¡Pero de esta manera podemos realizar muchas mas actualizaciones!

## Reflexiones

### Sobre la cantidad de ejemplos en la actualización

Pensemos que SGD está en un extremo, donde cada actualización de pesos considera un ejemplo de entrenamiento, y que el GD normalito está en otro extremo, donde cada actualización de pesos considera todos los ejemplos de entrenamiento. Podemos encontrar algoritmos entre estos dos extremos considerando $m$ ejemplos de entrenamiento con $1 < m < |\mathcal{D}_{\text{train}}|$. A esta modificación se le suele llamar *Descenso de Gradiente Estocástico por minilotes*.

### Sobre el orden de la iteración

Otras formas de variar el SGD es cambiando el orden en que iteramos los ejemplos de entrenamiento.  Una manera sería por ejemplo, iterando en orden aleatorio $\mathcal{D}_{\text{train}}$ en cada época.

¿Por qué esta modificación puede ser importante?

### Sobre el tamaño de paso

Entre más grande el tamaño de paso $\eta$, podemos llegar a la pérdida de entrenamiento mínima más rápido, pero también podemos obtener resultados inestables y no poder navegar la curva apropiadamente.

Entre más pequeño el tamaño de paso $\eta$, obtenemos mayor estabilidad, pero llegaremos al mínimo lentamente. Cuando $\eta = 0$, los pesos nunca son actualizados.

Una estrategia que podemos considerar es fijar el tamaño de paso inicialmente en $1$ y luego decrementarlo conforme realizamos las actualizaciones. Entre más iteraciones realicemos, mas pequeño se vuelve $\eta$. En general podemos considerar cualquier función decreciente sobre la cantidad de iteraciones que contenga la asociación $(1, 1)$, que representa que en la primera actualización se utiliza $\eta = 1$. Un ejemplo sería usar la función,
$$\eta = 1 / \sqrt{\#\text{ de actualización}}.$$

Modificaciones más sofisticadas al tamaño de paso pueden realizarse al considerar los datos de entrenamiento para ajustar $\eta$.

In [ ]:
def feta(n):
    return 1 / np.sqrt(n)

In [ ]:
plt.close()
plt.ioff()

updates = np.linspace(1, 100, 100)
decfunc = feta(updates)

fig = plt.figure()
ax = fig.add_subplot()
ax.plot(updates, decfunc, label = "$\\frac{1}{\\sqrt{\\#\\text{ de actualización}}}$")
ax.legend()

ax.set_xlabel("actualizaciones")
ax.set_ylabel("$\\eta$")
ax.set_xlim((0, 100))
ax.set_ylim((0, 1.0))


fig.canvas.header_visible = False
display(fig.canvas);

# Demostración

Primero generamos un conjunto de datos lo suficientemente grande como para que sea notable la mejora en velocidad de convergencia.

Fijamos un modelo real de los datos que no sea utilizado por los algoritmos pero nos permita corroborar el modelo aprendido por el algoritmo de optimización.

In [ ]:
trueW = np.array([1, 2, 3, 4, 5])

Ahora creamos una función para generar ejemplos de entrenamiento a partir de entradas aleatorias de acuerdo una distribución normal.  Las salidas son calculadas a partir de la entrada, los pesos reales y un ruido adicional también distribuido conforme a una distribución normal.

In [ ]:
def generate_example():
    x = np.random.randn(len(trueW))
    y = trueW.dot(x) + np.random.randn()
    return (x, y)

Generamos un millón de ejemplos de entrenamiento.

In [ ]:
Dtrain = [generate_example() for i in range(10**6)]

Corroboremos la distribución de entradas en $\mathcal{D}_{\text{train}}$.

In [ ]:
plt.close()
plt.ioff()

fig, axs = plt.subplots(
    len(trueW) + 1, 1,
    figsize = (5, 10),
    sharex = True,
    tight_layout = True,
)

for i, ax in enumerate(axs[:-1]):
    ax.hist([x[i] for x, _ in Dtrain], bins = 20, density = True, fc = "black")
    ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax = 1))
    ax.set_ylim((0, .75))

axs[-1].hist([y for _, y in Dtrain], bins = 20, density = True, fc = "green")
axs[-1].yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax = 1))
axs[-1].set_ylim((0, .75))


fig.canvas.header_visible = False
display(fig.canvas);

Preparamos las funciones auxiliares para experimentar con los algoritmos de optimización.

Vamos a trabajar con regresión, pero no incluímos el término bias $1$ en el vector de características.

In [ ]:
def phi(x):
    return np.array(x)

El vector de pesos inicial será $[0, \dots, 0]$.

In [ ]:
def init_weights():
    return np.zeros(len(trueW))

La predicción se realiza calculando el producto punto entre $\mathbf{w}$ y el vector de características.

In [ ]:
def predict(w, x):
    return w.dot(phi(x))

Utilizaremos la pérdida cuadrática.

$$\text{Loss}_2(x, y, \mathbf{w}) = (\underbrace{\underbrace{\mathbf{w}\cdot\phi(x)}_{\text{predicción}} - y}_{\text{residual}})^2$$

In [ ]:
def residual(x, y, w):
    return predict(w, x) - y

In [ ]:
def loss_2(x, y, w):
    return residual(x, y, w) ** 2

A partir de un conjunto de datos de entrenamiento, una función de pérdida y un vector de pesos. La pérdida de entrenamiento se implementa como la pérdida media sobre el conjunto de datos.

$$\text{TrainLoss}(\mathbf{w}) = \frac{1}{|\mathcal{D}_{\text{train}}|} \sum_{(x, y)\in\mathcal{D}_{\text{train}}} \text{Loss}(x, y, \mathbf{w})$$

In [ ]:
def train_loss(Dtrain, loss, w):
    examples = len(Dtrain)
    total = sum(loss(x, y, w) for x, y in Dtrain)
    return total / examples

Implementemos el gradiente de la pérdida cuadrática con respecto al vector de pesos $\mathbf{w}$.

$$\begin{aligned}
\nabla_{\mathbf{w}}\text{Loss}_2(x, y, \mathbf{w}) &=
\nabla_{\mathbf{w}} (\mathbf{w}\cdot\phi(x) - y)^2 \\
&= 2(\mathbf{w}\cdot\phi(x) - y)\nabla_{\mathbf{w}}(\mathbf{w}\cdot\phi(x) - y) \\
&= 2(\mathbf{w}\cdot\phi(x) - y)(\nabla_{\mathbf{w}}\mathbf{w}\cdot\phi(x) - \nabla_{\mathbf{w}} y) \\
&= 2(\mathbf{w}\cdot\phi(x) - y)\nabla_{\mathbf{w}}\mathbf{w}\cdot\phi(x) \\
&= 2(\mathbf{w}\cdot\phi(x) - y)\phi(x)
\end{aligned}$$

In [ ]:
def grad_loss_2(x, y, w):
    return 2 * (residual(x, y, w)) * phi(x)

A partir de un conjunto de datos de entrenamiento, el gradiente de una función de pérdida y un vector de pesos. El gradiente de la pérdida de entrenamiento se implementa como el promedio de los gradientes de pérdida sobre el conjunto de datos.

$$\nabla_{\mathbf{w}}\text{TrainLoss}(\mathbf{w}) = \frac{1}{|\mathcal{D}_{\text{train}}|} \sum_{(x, y)\in\mathcal{D}_{\text{train}}} \nabla_{\mathbf{w}}\text{Loss}(x, y, \mathbf{w})$$

In [ ]:
def grad_train_loss(Dtrain, grad_loss, w):
    examples = len(Dtrain)
    total = sum(grad_loss(x, y, w) for x, y in Dtrain)
    return total / examples

El algoritmo de descenso de gradiente (GD) realiza $T$ épocas actualizando los pesos tomando en cuenta a todos los datos de entrenamiento.

In [ ]:
def GD(Dtrain, loss, grad_loss, make_w = init_weights, eta = 0.1, T = 500):
    w_hist = []
    tl_hist = []
    w = make_w()
    for t in range(1, T + 1):
        tl = train_loss(Dtrain, loss, w)
        gtl = grad_train_loss(Dtrain, grad_loss, w)
        w_hist.append(w)
        tl_hist.append(tl)
        w = w - eta * gtl
    return w, (w_hist, tl_hist)

In [ ]:
%%time

gd_w, (gd_ws, gd_tls) = GD(Dtrain, loss_2, grad_loss_2, T = 20)

El algoritmo de descenso de gradiente estocástico (SGD) realiza $T$ épocas actualizando los pesos por cada ejemplo uno por uno.

In [ ]:
def SGD(Dtrain, loss, grad_loss, make_w = init_weights, eta_func = feta, T = 500):
    w_hist = []
    tl_hist = []
    w = make_w()
    n = 1
    for t in range(1, T + 1):
        w_hist.append(w)
        tl_hist.append(train_loss(Dtrain, loss, w))
        for x, y in Dtrain:
            l = loss(x, y, w)
            gl = grad_loss(x, y, w)
            eta = eta_func(n)
            w = w - eta * gl
            n += 1
    return w, (w_hist, tl_hist)

In [ ]:
%%time

sgd_w, (sgd_ws, sgd_tls) = SGD(Dtrain, loss_2, grad_loss_2, T = 20)

In [ ]:
plt.close()
plt.ioff()

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot()

epochs = list(range(1, len(sgd_tls) + 1))
ax.plot(epochs, gd_tls, label = "GD")
ax.plot(epochs, sgd_tls, label = "SGD")
ax.legend()
ax.set_title("Historia de TrainLoss")
ax.set_xlabel("época")
ax.set_ylabel("TrainLoss")
ax.set_xlim((epochs[0], epochs[-1]))

fig.canvas.header_visible = False
display(fig.canvas);